In [3]:
from qiskit.optimization.applications.ising import tsp
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axes

import pennylane as qml
from pennylane.operation import Tensor

In [127]:
nodes = 3
starting_node = 0

In [219]:

def get_graph(nodes):

    G = nx.Graph()
    G.add_nodes_from(np.arange(0, nodes, 1))

    # Create random positions in the graph. Distance will be calculated from positions
    # Note: Dwave and other solvers require a complete graph
    for i in range(nodes):
        G.nodes[i]['pos'] = (np.random.uniform(0, 10), np.random.uniform(0, 10))

    elist = set()
    for i in range(nodes):
        for t in range(i + 1,nodes):
            y1=G.nodes[i]['pos'][1]
            x1=G.nodes[i]['pos'][0]
            y2=G.nodes[t]['pos'][1]
            x2=G.nodes[t]['pos'][0]
            dist = np.sqrt(((x2-x1)**2)+((y2-y1)**2))
            _tuple = (i, t, dist)
            elist.add(_tuple)

    # tuple is (i,j,weight) where (i,j) is the edge
    G.add_weighted_edges_from(elist)

    return G

def get_cost_matrix(G, nodes):
    w = np.zeros([nodes,nodes])
    for i in range(nodes):
        for j in range(nodes):
            temp = G.get_edge_data(i,j,default=0)
            if temp != 0:
                w[i,j] = temp['weight']
    return w

def calculate_cost(cost_matrix, solution):
    cost = 0
    for i in range(len(solution)):
        a = i % len(solution)
        b = (i + 1) % len(solution)
        cost += cost_matrix[solution[a]][solution[b]]

    return cost

def binary_state_to_points_order(binary_state):
    points_order = []
    number_of_points = int(np.sqrt(len(binary_state)))
    for p in range(number_of_points):
        for j in range(number_of_points):
            if binary_state[(number_of_points) * p + j] == 1:
                points_order.append(j)
    return points_order

G = get_graph(nodes)
cost_matrix = get_cost_matrix(G, nodes)

In [220]:
coords = []
for node in G.nodes:
    coords.append(G.nodes[node]['pos'])

tsp_instance = tsp.TspData(name = "TSP", dim = len(G.nodes), coord = coords, w = cost_matrix)

qubitOp, offset = tsp.get_operator(tsp_instance)
print("Qubits needed: ", qubitOp.num_qubits)


Qubits needed:  9


# Now, the actual Pennylane

In [221]:
dev = qml.device('default.qubit', wires=qubitOp.num_qubits)
qubits = qubitOp.num_qubits

In [222]:
@qml.qnode(dev)
def circuit(params):
    qml.broadcast(qml.RX, list(range(qubitOp.num_qubits)), 'single', params[0])
    qml.broadcast(qml.RY, list(range(qubitOp.num_qubits)), 'single', params[1])
    qml.broadcast(qml.CNOT, list(range(qubitOp.num_qubits)), pattern="ring")
    return [qml.expval(qml.PauliZ(i)) for i in range(qubitOp.num_qubits)]

In [223]:
params = [2*np.pi * np.random.random_sample((qubits)), 2*np.pi * np.random.random_sample((qubits))]
params

[array([5.54010667, 4.35130431, 4.55690703, 3.14991399, 6.00725065,
        4.04630976, 2.66315981, 3.81008093, 0.12059442]),
 array([1.89485046, 4.14799267, 1.82261136, 3.88310546, 2.6940332 ,
        0.85120865, 1.87416313, 3.58119515, 3.71256305])]

In [224]:
res = circuit(params)
res

array([-2.98776830e-04, -4.43103193e-02, -1.70968342e-03, -1.26075836e-03,
        1.09358543e-03, -4.45359941e-04, -1.18106005e-04, -8.38724243e-05,
        7.00558057e-05])

In [225]:
def cost(params):
    exp_values = circuit(params)
    return np.square( np.dot(exp_values, np.ones(qubits) * -1) )

In [226]:
init_params = params
print(cost(init_params))

0.002214948094824717


In [227]:
opt = qml.GradientDescentOptimizer(stepsize=0.2)

# set the number of steps
steps = 40
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)
    print(params)
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Ansatz1: Optimized rotation angles: {}".format(params))


[array([5.53929784, 4.3536504 , 4.55723766, 3.14991415, 6.00725181,
       4.04633074, 2.66315561, 3.81008558, 0.12059494]), array([1.89222953, 4.14939212, 1.82240992, 3.88312345, 2.69403517,
       0.85122747, 1.87413723, 3.58119792, 3.71256582])]
Cost after step     1:  0.0021403
[array([5.53851504, 4.35593359, 4.55755723, 3.14991431, 6.00725291,
       4.04635102, 2.66315153, 3.8100901 , 0.12059545]), array([1.88967489, 4.15074859, 1.82221548, 3.88314082, 2.69403704,
       0.85124566, 1.87411212, 3.58120061, 3.71256851])]
Cost after step     2:  0.0020696
[array([5.53775694, 4.35815676, 4.55786633, 3.14991446, 6.00725396,
       4.04637063, 2.66314758, 3.81009448, 0.12059595]), array([1.88718353, 4.15206419, 1.82202765, 3.88315762, 2.69403883,
       0.85126324, 1.87408777, 3.58120322, 3.71257114])]
Cost after step     3:  0.0020025
[array([5.53702231, 4.36032262, 4.55816551, 3.14991461, 6.00725497,
       4.04638962, 2.66314374, 3.81009875, 0.12059643]), array([1.88475266, 4.15334

[array([5.5235151 , 4.40246238, 4.56361343, 3.14991729, 6.00727042,
       4.04673579, 2.66307163, 3.81018076, 0.12060598]), array([1.83657464, 4.17715329, 1.81857932, 3.88346874, 2.69406672,
       0.85159069, 1.87361949, 3.5812546 , 3.71262434])]
Cost after step    30:  0.0009861
[array([5.52314323, 4.403694  , 4.56376228, 3.14991736, 6.00727076,
       4.04674529, 2.66306958, 3.81018314, 0.12060627]), array([1.83513638, 4.17781845, 1.81849113, 3.88347679, 2.69406729,
       0.85159921, 1.87360687, 3.58125601, 3.71262585])]
Cost after step    31:  0.0009652
[array([5.52277894, 4.40490476, 4.56390806, 3.14991744, 6.00727108,
       4.0467546 , 2.66306758, 3.81018548, 0.12060655]), array([1.83372049, 4.17847056, 1.81840482, 3.88348467, 2.69406784,
       0.85160756, 1.87359449, 3.5812574 , 3.71262733])]
Cost after step    32:  0.0009451
[array([5.522422  , 4.40609525, 4.56405086, 3.14991751, 6.00727139,
       4.04676373, 2.66306561, 3.81018778, 0.12060682]), array([1.83232633, 4.17911

In [228]:
print(params)
print(circuit(params))
print(cost(params))

[array([5.52011075, 4.41390669, 4.56497468, 3.14991796, 6.0072733 ,
       4.04682286, 2.66305274, 3.81020285, 0.12060864]), array([1.8231281 , 4.1832629 , 1.81777497, 3.8835424 , 2.6940716 ,
       0.85166877, 1.87350292, 3.58126775, 3.71263844])]
[-2.17563811e-04 -2.67838004e-02 -9.61773728e-04 -7.08949307e-04
  6.14959951e-04 -2.50145931e-04 -6.61931616e-05 -4.70006311e-05
  3.92560722e-05]
0.0008054931350155965


In [229]:
@qml.qnode(dev)
def final_circ(params):
    qml.broadcast(qml.RX, list(range(qubitOp.num_qubits)), 'single', params[0])
    qml.broadcast(qml.RY, list(range(qubitOp.num_qubits)), 'single', params[1])
    qml.broadcast(qml.CNOT, list(range(qubitOp.num_qubits)), pattern="ring")
    return qml.probs(wires=range(qubitOp.num_qubits))

In [230]:
result = final_circ(params)
print("Different states: ", len(result))
print(min(result))
i, = np.where(np.isclose(result, min(result))) 
print(i)
print("Cost: ", cost(params))

Different states:  512
9.54566212706742e-07
[443]
Cost:  0.0008054931350155965


In [231]:
binary_result = list(map(int, bin(i[0])[2:]))
print(binary_result)
path_result = binary_state_to_points_order(list(binary_result))
path_result

[1, 1, 0, 1, 1, 1, 0, 1, 1]


[0, 1, 0, 1, 2, 1, 2]

## Pennylane VQE

In [232]:
dev = qml.device('default.qubit', wires=qubitOp.num_qubits)

def circuit(params, wires):
    #qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])

In [233]:
qubitOp.to_dict()['paulis']

[{'label': 'IIIIIIIIZ', 'coeff': {'real': -100002.44122189419, 'imag': 0.0}},
 {'label': 'IIIIZIIII', 'coeff': {'real': -100001.73958282053, 'imag': 0.0}},
 {'label': 'IIIIZIIIZ', 'coeff': {'real': 0.3781811300550017, 'imag': 0.0}},
 {'label': 'IIIIIIIZI', 'coeff': {'real': -100002.44122189419, 'imag': 0.0}},
 {'label': 'IIIZIIIII', 'coeff': {'real': -100001.73958282053, 'imag': 0.0}},
 {'label': 'IIIZIIIZI', 'coeff': {'real': 0.3781811300550017, 'imag': 0.0}},
 {'label': 'IIIIIIZII', 'coeff': {'real': -100002.44122189419, 'imag': 0.0}},
 {'label': 'IIIIIZIII', 'coeff': {'real': -100001.73958282053, 'imag': 0.0}},
 {'label': 'IIIIIZZII', 'coeff': {'real': 0.3781811300550017, 'imag': 0.0}},
 {'label': 'IZIIIIIII', 'coeff': {'real': -100002.66808019449, 'imag': 0.0}},
 {'label': 'IZIIIIIIZ', 'coeff': {'real': 0.8424298170353066, 'imag': 0.0}},
 {'label': 'ZIIIIIIII', 'coeff': {'real': -100002.66808019449, 'imag': 0.0}},
 {'label': 'ZIIIIIIZI', 'coeff': {'real': 0.8424298170353066, 'imag'

In [234]:
coeffs = []
obs = []
for i in qubitOp.to_dict()['paulis']:
    obs.append(i['label'])
    coeffs.append(i['coeff']['real'])

final_obs = []
for idx, observable in enumerate(obs):
    for sidx, s in enumerate(observable):
        if(sidx == 0): T = Tensor()
        if (s == 'I'):
            T = Tensor(T, qml.Identity(sidx))
        elif (s == 'Z'):
            T = Tensor(T, qml.PauliZ(sidx))
    final_obs.append(T)

H = qml.Hamiltonian(coeffs, final_obs)
print(H) 


(-100002.44122189419) [I0 I1 I2 I3 I4 I5 I6 I7 Z8]
+ (-100001.73958282053) [I0 I1 I2 I3 Z4 I5 I6 I7 I8]
+ (0.3781811300550017) [I0 I1 I2 I3 Z4 I5 I6 I7 Z8]
+ (-100002.44122189419) [I0 I1 I2 I3 I4 I5 I6 Z7 I8]
+ (-100001.73958282053) [I0 I1 I2 Z3 I4 I5 I6 I7 I8]
+ (0.3781811300550017) [I0 I1 I2 Z3 I4 I5 I6 Z7 I8]
+ (-100002.44122189419) [I0 I1 I2 I3 I4 I5 Z6 I7 I8]
+ (-100001.73958282053) [I0 I1 I2 I3 I4 Z5 I6 I7 I8]
+ (0.3781811300550017) [I0 I1 I2 I3 I4 Z5 Z6 I7 I8]
+ (-100002.66808019449) [I0 Z1 I2 I3 I4 I5 I6 I7 I8]
+ (0.8424298170353066) [I0 Z1 I2 I3 I4 I5 I6 I7 Z8]
+ (-100002.66808019449) [Z0 I1 I2 I3 I4 I5 I6 I7 I8]
+ (0.8424298170353066) [Z0 I1 I2 I3 I4 I5 I6 Z7 I8]
+ (-100002.66808019449) [I0 I1 Z2 I3 I4 I5 I6 I7 I8]
+ (0.8424298170353066) [I0 I1 Z2 I3 I4 I5 Z6 I7 I8]
+ (0.3781811300550017) [I0 I1 I2 I3 I4 Z5 I6 Z7 I8]
+ (0.3781811300550017) [I0 I1 I2 I3 Z4 I5 Z6 I7 I8]
+ (0.3781811300550017) [I0 I1 I2 Z3 I4 I5 I6 I7 Z8]
+ (0.49161028021195696) [I0 Z1 I2 I3 I4 Z5 I6 I7 I8]
+ (0

In [235]:
cost_fn = qml.VQECost(circuit, H, dev)

In [236]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
np.random.seed(0)
params = np.random.normal(0, np.pi, (qubitOp.num_qubits, 3))
print(params)

[[ 5.54193389  1.25713095  3.07479606]
 [ 7.03997361  5.86710646 -3.07020901]
 [ 2.98479079 -0.47550269 -0.32427159]
 [ 1.28993324  0.45252622  4.56873497]
 [ 2.39087053  0.38225334  1.39443747]
 [ 1.04826882  4.69378784 -0.64452369]
 [ 0.98353119 -2.6832209  -8.02045405]
 [ 2.05340338  2.71570641 -2.33158018]
 [ 7.13064445 -4.56902452  0.14375462]]


In [237]:
max_iterations = 100
conv_tol = 1e-06

prev_energy = cost_fn(params)
for n in range(max_iterations):
    params = opt.step(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)

    if n % 5 == 0:
        print('Iteration = {:},  Cost = {:.8f},  Convergente = {'
              ':.8f} Ha'.format(n, energy, conv))
        print("Parameters:")
        print(params)

    if conv <= conv_tol:
        break

    prev_energy = energy

print("===================")
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('Cost: ',energy)
print('Final circuit parameters = \n', params)

Iteration = 0,  Cost = -44405.94955675,  Convergente = 45382.12094243 Ha
Parameters:
[[ 5.54193389e+00  2.78811379e+03  3.07479606e+00]
 [ 7.03997361e+00  3.30916058e+03 -3.07020901e+00]
 [ 2.98479079e+00  5.92741829e+04 -3.24271587e-01]
 [ 1.28993324e+00 -1.63886430e+04  4.56873497e+00]
 [ 2.39087053e+00 -1.04318295e+04  1.39443747e+00]
 [ 1.04826882e+00 -9.46469189e+03 -6.44523694e-01]
 [ 9.83531192e-01  1.75169344e+04 -8.02045405e+00]
 [ 2.05340338e+00 -1.14036618e+04 -2.33158018e+00]
 [ 7.13064445e+00 -5.81420151e+04  1.43754622e-01]]
Iteration = 5,  Cost = 429019.83323319,  Convergente = 185741.61887101 Ha
Parameters:
[[ 5.54193389e+00 -5.72654989e+04  3.07479606e+00]
 [ 7.03997361e+00  1.93283648e+04 -3.07020901e+00]
 [ 2.98479079e+00  6.73673059e+04 -3.24271587e-01]
 [ 1.28993324e+00 -7.57828500e+02  4.56873497e+00]
 [ 2.39087053e+00  1.10693876e+05  1.39443747e+00]
 [ 1.04826882e+00  6.30556742e+04 -6.44523694e-01]
 [ 9.83531192e-01  6.84070600e+04 -8.02045405e+00]
 [ 2.0534033

Iteration = 75,  Cost = 124931.59016928,  Convergente = 228568.35866622 Ha
Parameters:
[[ 5.54193389e+00 -1.27786784e+05  3.07479606e+00]
 [ 7.03997361e+00 -4.13936985e+04 -3.07020901e+00]
 [ 2.98479079e+00 -6.81878959e+05 -3.24271587e-01]
 [ 1.28993324e+00 -2.95726171e+05  4.56873497e+00]
 [ 2.39087053e+00  8.12781144e+05  1.39443747e+00]
 [ 1.04826882e+00  2.04935420e+04 -6.44523694e-01]
 [ 9.83531192e-01 -1.04924850e+05 -8.02045405e+00]
 [ 2.05340338e+00  2.21454926e+05 -2.33158018e+00]
 [ 7.13064445e+00  5.45448520e+05  1.43754622e-01]]
Iteration = 80,  Cost = -10476.60074461,  Convergente = 364009.49811926 Ha
Parameters:
[[ 5.54193389e+00 -9.92441198e+04  3.07479606e+00]
 [ 7.03997361e+00 -5.33126450e+04 -3.07020901e+00]
 [ 2.98479079e+00 -6.37295627e+05 -3.24271587e-01]
 [ 1.28993324e+00 -3.95972135e+05  4.56873497e+00]
 [ 2.39087053e+00  7.73195064e+05  1.39443747e+00]
 [ 1.04826882e+00  9.76151096e+04 -6.44523694e-01]
 [ 9.83531192e-01 -1.42114341e+05 -8.02045405e+00]
 [ 2.0534

In [238]:
@qml.qnode(dev)
def final_circ(params):
    circuit(params, range(qubitOp.num_qubits))
    return qml.probs(wires=range(qubitOp.num_qubits))

In [249]:
result = final_circ(params)
print(result)

[1.83762197e-13 2.50242801e-13 2.29756743e-14 3.12877033e-14
 6.62150892e-16 9.01700659e-16 8.27883180e-17 1.12739078e-16
 3.34702449e-06 4.55789493e-06 4.18476410e-07 5.69870794e-07
 1.20603437e-08 1.64234769e-08 1.50789734e-09 2.05341719e-09
 4.34279848e-12 5.91391524e-12 5.42977417e-13 7.39413177e-13
 1.56484192e-14 2.13096290e-14 1.95651220e-15 2.66432978e-15
 7.90992549e-05 1.07715403e-04 9.88973110e-06 1.34675905e-05
 2.85018590e-07 3.88131245e-07 3.56356986e-08 4.85278104e-08
 3.66700129e-12 4.99363138e-12 4.58482911e-13 6.24350655e-13
 1.32133171e-14 1.79935674e-14 1.65205289e-15 2.24972464e-15
 6.67903591e-05 9.09534541e-05 8.35075743e-06 1.13718543e-05
 2.40665907e-07 3.27732862e-07 3.00903100e-08 4.09762378e-08
 8.66611732e-11 1.18013036e-10 1.08351931e-11 1.47550972e-11
 3.12266473e-13 4.25236737e-13 3.90424847e-14 5.31670871e-14
 1.57843710e-03 2.14947649e-03 1.97351018e-04 2.68747721e-04
 5.68758728e-06 7.74521527e-06 7.11115532e-07 9.68379492e-07
 1.30612697e-11 1.778651

In [262]:
print("Different states: ", len(result))
print(min(result))
idx = np.where(result == min(result))
idx = '{0:09b}'.format(idx[0][0])
print(idx)
print("Cost: ", cost_fn(params))

Different states:  512
7.179025780891749e-17
010100110
Cost:  118467.22984988677


In [268]:
binary_result = list(map(int, idx))
path_result = binary_state_to_points_order(list(binary_result))
path_result

[1, 0, 0, 1]